# An Age Guessing Game

Open in Google Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.png)](https://colab.research.google.com/github/febse/stat2025/blob/main/01-Introduction.ipynb)


In the following game we will try to guess the age of the person in the image. The images are taken from the [UTKFace dataset](https://susanqq.github.io/UTKFace/), a large-scale face dataset. The age of the persons in our game ranges from 15 to 90 years.

Scan the following QR code to open the game in your mobile device or click here:

[https://uni-sofia.vercel.app/surveys/UKTfaces](https://uni-sofia.vercel.app/surveys/UKTfaces).

![](./figures/qr_code_uktfaces.png)

## Data Import

Now that we have played the game, let's import the data and see what we can learn from it. The following block of code will download the data, restructure it and display the first few rows. In these introductory classes you do not need to understand the code, but you can run it and see the results.

The data consists of the following columns:

- **FileId** (character): The name of the image file.
- **UserId** (character): A unique identifier for the user.
- **Guess** (numeric): The age guessed by the user.
- **Age** (numeric): The actual age of the person in the image.
- **Gender** (character): The gender of the person in the image.
- **Position** (numeric): The position of the image in the game (0-16), 0 being the first image shown to the user, 1 the second, and so on. The images were shown in random order except for the first image which was always the same.
- **TimeStart** (numeric): The time when the user saw the image (milliseconds from 1970-01-01).
- **TimeEnd** (numeric): The time when the user submitted the guess (milliseconds from 1970-01-01).


Each row corresponds to a guess made by a user for an image.

In [1]:
#| echo: false

import pandas as pd
import numpy as np

# NOTE: you can ignore this code, it's just for data preparation

# Imports the age guessing data from the database,
# restructures it and merges it with the game data.

dt_raw = pd.read_json("https://get-uktfaces-data-suq3dizvtq-uc.a.run.app")
dt = pd.melt(
    dt_raw.reset_index(names="FileId"), 
    id_vars=["FileId"],
    var_name="UserId",
    value_name="Data"
)
dt_mesurements = pd.json_normalize(dt['Data']).rename(columns=lambda x: x[0].upper() + x[1:])
dt = pd.concat([dt.drop(columns=["Data"]), dt_mesurements], axis=1)

game = pd.read_json("https://get-uktfaces-game-suq3dizvtq-uc.a.run.app").rename(columns={"id": "FileId", "age": "Age", "race_f": "Race", "gender": "Gender"})
dt = dt.merge(game, on="FileId", how="left")[["FileId", "UserId", "Guess", "Age", "Race", "Gender", "Position", "TimeStart", "TimeEnd"]].copy()

# Prints the first 5 rows of the data frame so that we can check if we imported the data correctly
dt.head()

,FileId,UserId,Guess,Age,Race,Gender,Position,TimeStart,TimeEnd
0,72_1_0_20170110141531648_jpg,2TkJQyslSFb5GXbdv1aCc2QAZO53,58,72,White,F,10,1728456243117,1728456250343
1,62_0_0_20170111210223707_jpg,2TkJQyslSFb5GXbdv1aCc2QAZO53,70,62,White,M,9,1728456238837,1728456243116
2,75_0_1_20170111205346848_jpg,2TkJQyslSFb5GXbdv1aCc2QAZO53,57,75,Other,M,7,1728456228654,1728456235495
3,21_1_2_20170105183505385_jpg,2TkJQyslSFb5GXbdv1aCc2QAZO53,19,21,Other,F,16,1728456288579,1728456295650
4,28_0_1_20170112211810813_jpg,2TkJQyslSFb5GXbdv1aCc2QAZO53,30,28,Other,M,11,1728456250345,1728456254708


## First Steps with Python

Before we start working with the data, let's make sure that we understand how to use Python and the Jupyter Notebook. The following block of code will show you how to perform basic arithmetic operations in Python, print the results, and store them in variables.


:::{.callout-note}
## Running the Python Code

To run the code in the following cells, click on the cell and press `Ctrl+Enter` or click on the `Run` button (upper left side of the cell).

After running the cell, check the output below the cell. The last statement in the cell will be printed. If you want to print other values, use the `print()` function.
:::

:::{.callout-note}
## Variables in Visual Studio Code

Visual Studio Code allows you to see the values of the variables that you have defined in the code. Find the `Variables` tab on the top side of the notebook window and click on it. You will see the variables that you have defined and their values.

:::

:::{.callout-note}
## Variables in Google Colab

Google Colab also allows you to see the values of the variables that you have defined in the code. Click on the `{x}` icon on the right side of page.

:::




## Python as a Calculator

Python can be used as a calculator to perform basic arithmetic operations. You can use the following operators:

- Addition: `+`
- Subtraction: `-`
- Multiplication: `*`
- Division: `/`
- Exponentiation: `**`

You can also use parentheses `()` to group operations and control the order of evaluation.

Let's try some calculations in the next code cell.

In [ ]:
# Addition

5  + 6

11

In [ ]:
# Subtraction

10 - 4

2.0

In [12]:
# Multiplication

4 * 2.5

10.0

In [ ]:
# Division

8 / 2

In [13]:
# Exponents

2 ** 3

8

## Variables

You can store values in variables using the assignment operator `=`. For example, you can store the result of a calculation in a variable and use it later in your code. This feature is very powerful because it allows you to reuse values without having to recalculate them.

In [16]:
# Store a value in a variable

x = 10
x

10

In [17]:
y = 5
y

5

In [18]:
x_times_y = x * y
x_times_y

50

## Exercise


In [ ]:
# Create a new variable called a that is equal to 7


In [ ]:
# Create a new variable called a_div_5 that is equal to a divided by 5


## Data Description

The first thing we want to know about a new dataset is its size. How many rows and columns does it have? We can use the `shape` attribute of the DataFrame to find out.

In [3]:
# Data shape is a tuple (think of it as a list of numbers for now): the first number is the number of rows, the second number is the number of columns
dt.shape

(782, 9)

In [4]:
# You can access the individual elements of the tuple using square brackets

# The first number is the number of rows
dt.shape[0]
print("There are", dt.shape[0], "rows in the dataset.")

# The second number is the number of columns
dt.shape[1]
print("There are", dt.shape[1], "columns in the dataset.")

There are 782 rows in the dataset.
There are 9 columns in the dataset.


In [5]:
# What are the columns in the dataset?

# The columns are stored in the columns attribute of the DataFrame. Again, you will a list of column names (technically a pandas Index object, but for now you can think of it as a list)
dt.columns


Index(['FileId', 'UserId', 'Guess', 'Age', 'Race', 'Gender', 'Position',
       'TimeStart', 'TimeEnd'],
      dtype='object')

In [6]:
# How many unique users are there in the dataset? You can access the columns in the using the brackets notation

# The unique function returns the unique values in a Series

dt["UserId"].unique()

array(['2TkJQyslSFb5GXbdv1aCc2QAZO53', '4W0FEKDKyjNZUuCf9uBNs8rfjjq1',
       '4sgOBUwOCshVe7lLy6I9fnRi8KY2', '70VWATB9KdbMwFQOciduTEmg0k73',
       '8Dv06RmVjaVJE4ZFNRMK5f1oQOp1', '8fPccGO10Bb2la4FkBqgl1rVWHg1',
       '8jKBM5SOv0fOly6vCBdHXtxBPQ23', '9j9Y0NUvW0MJRsoa266Y1kMZSuY2',
       'CPjrqG93VHSL9IGx30p4ZSXd4Qr1', 'CbS6ZrS8dyd0awFJRvhc7OBK5YA2',
       'DuGjUzQlaUOS2bOkD2nzCq3TaMI3', 'GmKPuKAbgoYQCPmlg8RwPNsBHM23',
       'HqIR4yBnDLax4jjFITVp6hVvqx52', 'Lec4YA560vapZcD5fIP8MJm5sl42',
       'MxAZfVjX3eZRJXHxkdj8TrkCldD2', 'NPtpMro4nOOJTqTNpt8PPHmbiFk2',
       'OBczO7vxLvN2FiLBkqnQvCM8NdF2', 'PVb3jSAxyMXAuQ2PeIpdqQ0ILp53',
       'PphGn6oTxNPO1u8YiwrruLNTzXE2', 'QEskanUwcUNT2gyBQFvSaVHeI4D3',
       'SQC3UTi7a1P7CGyqhy7mBe8Zgmj2', 'Tvwjrx7GfaaBRYS6TGadjBjO9HF2',
       'TzOCmmyl29OjE7JyQjqJqg7lD3u1', 'UCEzPzCCb7YfusqZVuUmGPIMFVV2',
       'UpgLIyOuiVSd3y8VQ2gnJ6ViCxw2', 'VeyVpTOLy7NR0V5ZCnbLipceCEG2',
       'W8MAPl0tmKbJAJ3abhcnQrqG5no1', 'WLi6ASLPTccgRUo1KG6ThcH80Nx1',
      

In [7]:
dt["UserId"].nunique()

46

In [9]:
# Sometimes it is useful to sort the unique values. The np.sort function sorts the values in ascending order (from the smallest to the largest)
# The function is part of the numpy library, which is why we have imported it at the beginning of the notebook (import numpy as np)

np.sort(dt["UserId"].unique())

array(['2TkJQyslSFb5GXbdv1aCc2QAZO53', '4W0FEKDKyjNZUuCf9uBNs8rfjjq1',
       '4sgOBUwOCshVe7lLy6I9fnRi8KY2', '70VWATB9KdbMwFQOciduTEmg0k73',
       '8Dv06RmVjaVJE4ZFNRMK5f1oQOp1', '8fPccGO10Bb2la4FkBqgl1rVWHg1',
       '8jKBM5SOv0fOly6vCBdHXtxBPQ23', '9j9Y0NUvW0MJRsoa266Y1kMZSuY2',
       'CPjrqG93VHSL9IGx30p4ZSXd4Qr1', 'CbS6ZrS8dyd0awFJRvhc7OBK5YA2',
       'DuGjUzQlaUOS2bOkD2nzCq3TaMI3', 'GmKPuKAbgoYQCPmlg8RwPNsBHM23',
       'HqIR4yBnDLax4jjFITVp6hVvqx52', 'Lec4YA560vapZcD5fIP8MJm5sl42',
       'MxAZfVjX3eZRJXHxkdj8TrkCldD2', 'NPtpMro4nOOJTqTNpt8PPHmbiFk2',
       'OBczO7vxLvN2FiLBkqnQvCM8NdF2', 'PVb3jSAxyMXAuQ2PeIpdqQ0ILp53',
       'PphGn6oTxNPO1u8YiwrruLNTzXE2', 'QEskanUwcUNT2gyBQFvSaVHeI4D3',
       'SQC3UTi7a1P7CGyqhy7mBe8Zgmj2', 'Tvwjrx7GfaaBRYS6TGadjBjO9HF2',
       'TzOCmmyl29OjE7JyQjqJqg7lD3u1', 'UCEzPzCCb7YfusqZVuUmGPIMFVV2',
       'UpgLIyOuiVSd3y8VQ2gnJ6ViCxw2', 'VeyVpTOLy7NR0V5ZCnbLipceCEG2',
       'W8MAPl0tmKbJAJ3abhcnQrqG5no1', 'WLi6ASLPTccgRUo1KG6ThcH80Nx1',
      

In [ ]:
# Replace the string between the quotes with your own unique identifier
# (look up your id on the results page of the game)

my_id = "dLeiDreuBXPDO5EjgXEt8Q514mw2"


42

## Selecting Subsets of the Data

There are multiple ways to choose a part of the data. We can select rows by their index, columns by their name, or both. We can also use logical conditions to filter the data.
Here we want to select only the rows belonging to one of the users.

In [ ]:
# The == operator checks if two values are equal. It returns a boolean value (True or False). In this case, we are checking if the UserId is equal to "2TkJQyslSFb5GXbdv1aCc2QAZO53"
# for each row in the UserId column of the dataframe dt. As the dataframe is quite large, you will see the first 5 and the last 5 rows.

dt["UserId"] == my_id

0       True
1       True
2       True
3       True
4       True
       ...  
709    False
710    False
711    False
712    False
713    False
Name: UserId, Length: 714, dtype: bool

In [ ]:
# We can call the .sum() method of the result to count the number of True values. This will give us the number of rows where the UserId is equal to "2TkJQyslSFb5GXbdv1aCc2QAZO53"

(dt["UserId"] == my_id).sum()

17

In [ ]:
# Putting this boolean series in square brackets returns only the rows where the condition is True

mydt = dt[dt["UserId"] == my_id].copy()
mydt

,FileId,UserId,Guess,Age,Race,Gender,Position,TimeStart,TimeEnd
68,72_1_0_20170110141531648_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,69,72,White,F,7,1728466078201,1728466083168
69,62_0_0_20170111210223707_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,72,62,White,M,4,1728466058849,1728466065965
70,75_0_1_20170111205346848_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,69,75,Other,M,14,1728466118959,1728466125058
71,21_1_2_20170105183505385_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,32,21,Other,F,9,1728466093455,1728466098292
72,28_0_1_20170112211810813_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,36,28,Other,M,12,1728466106791,1728466112456
73,30_0_4_20170117202914440_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,43,30,Other,M,0,1728465970667,1728466037743
74,62_1_0_20170110175644800_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,68,62,White,F,5,1728466065967,1728466075283
75,16_0_0_20170110231841292_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,17,16,White,M,16,1728466134829,1728466141246
76,18_1_0_20170109214608184_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,22,18,White,F,6,1728466075284,1728466078199
77,75_0_3_20170111202756116_jpg,8jKBM5SOv0fOly6vCBdHXtxBPQ23,57,75,Other,M,1,1728466037744,1728466045288


:::{#ex-mydata}
## Select Your Data

Create a new DataFrame called `mydt` containing only the rows where the `UserId` is equal to your unique identifier (it is shown on the results page of the game).
To avoid typing the whole identifier, look at the sorted unique values of the `UserId` and copy the one that corresponds to you.

:::

## Counting Things

A lot of the time we will want to compute some simple counts. In our example, we may want to count the number of guesses made by each user, the number of guesses that overestimated the age, or the number of correct guesses. We can accomplish this using logical conditions and the `sum` method of series (or the `np.sum` function).

:::{#ex-counting-overestimations}

## Counting Overestimations

How many times did you overestimate the age of the person in the images?

:::

In [15]:
# Creates a new column called "Overestimates" that is True if the "Guess" is greater than the "Age" and False otherwise
mydt["Overestimates"] = (mydt["Guess"] > mydt["Age"])

# Creates a new column called "Underestimates" that is True if the "Guess" is less than the "Age" and False otherwise
mydt["Underestimates"] = (mydt["Guess"] < mydt["Age"])

# Creates a new column called "NoError" that is True if the "Guess" is equal to the "Age" and False otherwise
mydt["NoError"] = (mydt["Guess"] == mydt["Age"])

# Sums the values in the "Overestimates" column to get the number of overestimates
mydt["NoError"].sum()

2

In [16]:
# Exercise: In how many images did you underestimate the age?
# Write your code here


In [17]:
# Why does using the .sum() method actually work here?

x_logical = np.array([True, False, True, True, False])
x_logical

array([ True, False,  True,  True, False])

In [18]:
# Test it by changing the values (True/False) in the assignment of x_logical and running the cell again (you also need to run the cell with the definition of x_logical)
x_logical.sum()

3

In [19]:
# Exercise: How many female images have you looked at during the guessing game?



In [20]:
# Exercise: How many male images have you guessed?



In [21]:
# Compare your results with the output of the following code

mydt["Gender"].value_counts()

Gender
M    9
F    8
Name: count, dtype: int64